### 0. Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid

from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import pywt
import copy
import json

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/WaveletFrequencyDecomposed_CNN_Transformer/data/train_data.csv')
df.set_index('timestamp', inplace=True)

### 1. Data Preprocessing

##### 1.1 Wavelet Frequency Decompose Correlation

In [ ]:
def wavelet_decomposed_corr(df, input_window_width=30, label_window_width=10, wavelet='db4', level=3):
    X, Y = [], []
    data = df.values

    for t in range(input_window_width, len(df) - 1):
        window_data = data[t-input_window_width : t]

        low_band, mid_band, high_band = [], [], []
        for i in range(window_data.shape[1]):
            comod_per_widow = window_data[:, i]
            coeffs = pywt.wavedec(comod_per_widow, wavelet, level=level)
            cA3, cD3, _, cD1 = coeffs
            low_band.append(cA3)
            mid_band.append(cD3)
            high_band.append(cD1)

        corr_low = np.corrcoef(low_band)
        corr_mid = np.corrcoef(mid_band)
        corr_high = np.corrcoef(high_band)

        corr_tensor = torch.tensor(np.stack([corr_low, corr_mid, corr_high]), dtype=torch.float32)
        X.append(corr_tensor)

        label_window = data[t : t+label_window_width]
        corr_next = np.corrcoef(label_window.T)
        Y.append(torch.tensor(corr_next, dtype=torch.float32))


    return torch.stack(X), torch.stack(Y)

In [ ]:
X_tensor, Y_tensor = wavelet_decomposed_corr(df)

total_size = len(X_tensor)
train_size = int(total_size * 0.8)
val_size   = int(total_size * 0.1)

test_size  = total_size - train_size - val_size

X_train = X_tensor[:train_size]
Y_train = Y_tensor[:train_size]

X_val = X_tensor[train_size:train_size + val_size]
Y_val = Y_tensor[train_size:train_size + val_size]

X_test = X_tensor[train_size + val_size:]
Y_test = Y_tensor[train_size + val_size:]

train_ds = TensorDataset(X_train, Y_train)
val_ds   = TensorDataset(X_val, Y_val)
test_ds  = TensorDataset(X_test, Y_test)

/usr/local/lib/python3.11/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  warnings.warn(


### 2. Modeling

##### 2.1 Model Structure Setting

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]

        return x

In [ ]:
class CorrPredictorTransformer(nn.Module):
    def __init__(
            self,
            num_channels=3,
            nhead=4,
            num_layers=2,
            dim_feedforward=256,
            activation='relu',
            d_model=64,
        ):
        super().__init__()

        self.positional_encoding = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            activation=activation,
            batch_first=True
        )

        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, 64)
        )

    def forward(self, x):
        """
        x: (B, 3, 8, 8)
        output: (B, 8, 8)
        """
        B, C, H, W = x.shape

        x, _ = x.max(dim=1)
        x = x.view(B, -1)
        x = x.unsqueeze(0)

        x = self.positional_encoding(x)
        x = self.transformer(x)
        x = x.squeeze(0)

        x = self.fc(x)
        x = x.view(-1, 8, 8)
        return x


##### 2.2 Training

In [ ]:
def train_model(model, train_loader, val_loader, optimizer_name='Adam', lr=5e-4, epochs=70, device='cuda'):
    model.to(device)

    # Optimizer 선택
    if optimizer_name == 'Adam':
        opt = torch.optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name == 'RMSprop':
        opt = torch.optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_name == 'AdamW':
        opt = torch.optim.AdamW(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")

    # Loss & LR Scheduler
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        opt, mode='min', factor=0.5, patience=5, verbose=True
    )

    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()

            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            opt.step()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                val_loss += criterion(model(xb), yb).item()
        val_loss /= len(val_loader)

        # 스케줄러 적용
        scheduler.step(val_loss)

    return val_loss

In [ ]:
model_save_path = '/content/drive/MyDrive/WaveletFrequencyDecomposed_CNN_Transformer/best_model'

In [ ]:
param_grid = {
    'nhead': [2, 4],
    'num_layers': [2, 4],
    'dim_feedforward': [256, 512],
    'activation': ['relu', 'gelu'],
    'lr': [0.001, 5e-4],
    'optimizer': ['Adam', 'RMSprop', 'AdamW'],
    'batch_size': [64, 128, 256, 512],
}

best_loss = float('inf')
best_config = None

for config in tqdm(ParameterGrid(param_grid)):
    train_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=False)
    val_loader   = DataLoader(val_ds,   batch_size=config['batch_size'], shuffle=False)

    model = CorrPredictorTransformer(
        d_model=64,  # 고정
        nhead=config['nhead'],
        num_layers=config['num_layers'],
        dim_feedforward=config['dim_feedforward'],
        activation=config['activation'],
    )

    loss = train_model(
        model, train_loader, val_loader,
        optimizer_name=config['optimizer'],
        lr=config['lr'],
        device=device
    )

    print(f"Config: {config}, Loss: {loss:.4f}")

    if loss < best_loss:
        best_loss = loss
        best_config = config
        torch.save(model.state_dict(), f"{model_save_path}/best_model_window10per30_WT_weights.pth")
        with open(f'{model_save_path}/best_model_window10per30_WT_config.json', 'w') as f:
            json.dump(best_config, f, indent=4)

print(f"\n✅ Best Config: {best_config}")
print(f"✅ Best Loss: {best_loss:.4f}")


  0%|          | 0/384 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Config: {'activation': 'relu', 'batch_size': 64, 'dim_feedforward': 256, 'lr': 0.001, 'nhead': 2, 'num_layers': 2, 'optimizer': 'Adam'}, Loss: 0.0662


  1%|          | 2/384 [00:31<1:35:25, 14.99s/it]

Config: {'activation': 'relu', 'batch_size': 64, 'dim_feedforward': 256, 'lr': 0.001, 'nhead': 2, 'num_layers': 2, 'optimizer': 'RMSprop'}, Loss: 0.0657


  1%|          | 3/384 [00:43<1:27:28, 13.78s/it]

...

Config: {'activation': 'gelu', 'batch_size': 512, 'dim_feedforward': 512, 'lr': 0.0005, 'nhead': 4, 'num_layers': 4, 'optimizer': 'Adam'}, Loss: 0.0685


100%|█████████▉| 383/384 [52:56<00:03,  3.17s/it]

Config: {'activation': 'gelu', 'batch_size': 512, 'dim_feedforward': 512, 'lr': 0.0005, 'nhead': 4, 'num_layers': 4, 'optimizer': 'RMSprop'}, Loss: 0.0662


100%|██████████| 384/384 [52:59<00:00,  8.28s/it]

Config: {'activation': 'gelu', 'batch_size': 512, 'dim_feedforward': 512, 'lr': 0.0005, 'nhead': 4, 'num_layers': 4, 'optimizer': 'AdamW'}, Loss: 0.0693

✅ Best Config: {'activation': 'gelu', 'batch_size': 64, 'dim_feedforward': 256, 'lr': 0.0005, 'nhead': 2, 'num_layers': 4, 'optimizer': 'Adam'}
✅ Best Loss: 0.0634


##### 2.3 Test

In [ ]:
def evaluate_model(model, test_loader, device='cuda'):
    model.eval()
    model.to(device)

    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            preds.append(pred)
            targets.append(yb)

    preds_tensor = torch.cat(preds, dim=0)      # (B*T, 8, 8)
    targets_tensor = torch.cat(targets, dim=0)  # (B*T, 8, 8)

    return preds_tensor, targets_tensor

with open(f'{model_save_path}/best_model_window10per30_WT_config.json', 'r') as f:
    best_config = json.load(f)

best_model = CorrPredictorTransformer(
    d_model=64,
    nhead=best_config['nhead'],
    num_layers=best_config['num_layers'],
    dim_feedforward=best_config['dim_feedforward'],
    activation=best_config['activation']
)

best_model.load_state_dict(torch.load(f"{model_save_path}/best_model_window10per30_WT_weights.pth"))

test_loader = DataLoader(test_ds, batch_size=best_config['batch_size'], shuffle=False)

preds_tensor, targets_tensor = evaluate_model(best_model, test_loader, device=device)

torch.save({
    'preds': preds_tensor,
    'targets': targets_tensor
}, f"{model_save_path}/best_model_window10per30_WT_result.pt")


In [ ]:
# Performance metrics

preds_flat = preds_tensor.view(preds_tensor.size(0), -1).cpu().numpy()
targets_flat = targets_tensor.view(targets_tensor.size(0), -1).cpu().numpy()

mse = mean_squared_error(targets_flat, preds_flat)
mae = mean_absolute_error(targets_flat, preds_flat)
rmse = np.sqrt(mse)

# frobenius_loss
cos_sim = cosine_similarity(targets_flat, preds_flat)
mean_cos_sim = np.diag(cos_sim).mean()

# frobenius_loss
diff = preds_tensor - targets_tensor
frobenius_per_sample = torch.norm(diff, p='fro', dim=(1, 2))
mean_frobenius = frobenius_per_sample.mean().item()

print(f"\n📊 Evaluation Results:")
print(f"MSE               : {mse:.5f}")
print(f"MAE               : {mae:.5f}")
print(f"RMSE              : {rmse:.5f}")
print(f"Cosine Similarity : {mean_cos_sim:.5f}")
print(f"Frobenius Norm    : {mean_frobenius:.5f}")


📊 Evaluation Results:
MSE               : 0.07675
MAE               : 0.18852
RMSE              : 0.27704
Cosine Similarity : 0.93675
Frobenius Norm    : 1.96901
